# Define Imports and Set Up GPU

In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import os
from time import sleep
#from skimage.transform import resize
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Set Up Normalization

In [2]:
def norma2(input_data):
    """Normalize all data based on their own max values
    
    Parameters
    ==========
    
    input_data - (np.array)
        the full array of all 2D images to normalize based upon their own np.max value
        
        
    Returns
    =======
    The normalized data
    
    """
    
    maxies = np.max(input_data, axis=(1,2))
    if np.count_nonzero(maxies) != len(input_data):
        print('Error - Divide By Zero Found')
    output = np.divide(input_data, maxies[:,None, None])

    
    return output

# Import Trained Model

In [3]:
# Basic TF Imports
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model


# MODELS AND OPTIMIZERS
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Nadam

Nc = 64
n_labels = 3

base_model=VGG16(input_shape=(Nc,Nc,3),
                 include_top=False, )

#base_model.layers.pop() #Pop out the current logistic layer
tmp = base_model.output

#Add a fully connected layer
tmp=Flatten()(tmp)

# and a logistic layer -- let's say we have 200 classes
predictions = Dense(n_labels, activation='softmax')(tmp)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

#Fix the first 17 layers
for layer in model.layers[:15]:
    layer.trainable = False 
    

opto = Adam() 
    
model.compile(optimizer=opto, loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.load_weights('../trained_model-VGG16/VGG16_trained.hdf5')

# Test Model On Testing Data

In [5]:
# Load in the already split Training and Test Set2
for i in ['2', '3', '14']:


    xTest_testing = np.load(f'../testing_data/new_test_data/test_data_64/noise_test_c{i}_64x64_dataset_01.npy', allow_pickle=True)
    yTest_testing = np.load(f'../testing_data/new_test_data/test_data_64/noise_test_c{i}_64x64_labels_01.npy', allow_pickle=True)

    # used for padding
    value = 0


    # Section out Perfect - Edge - Screw - Testing Testing - C2 Data
    perf_dat = []
    edge_dat = []
    screw_dat = []

    perf_lab = []
    edge_lab = []
    screw_lab = []


    #for idx, labels in tqdm(enumerate(yTest_testing), total=len(yTest_testing)):
    for idx, labels in enumerate(yTest_testing):
        if labels[0] == 1:
            perf_lab.append(labels)
            perf_dat.append(xTest_testing[idx])

        elif labels[1] == 1:
            edge_lab.append(labels)
            edge_dat.append(xTest_testing[idx])

        elif labels[2] == 1:
            screw_lab.append(labels)
            screw_dat.append(xTest_testing[idx])

    perf_dat = np.asarray(perf_dat)
    edge_dat = np.asarray(edge_dat)
    screw_dat = np.asarray(screw_dat)

    perf_lab = np.asarray(perf_lab)
    edge_lab = np.asarray(edge_lab)
    screw_lab = np.asarray(screw_lab)


    X_test_testing = np.vstack((perf_dat, edge_dat,screw_dat))
    y_test_testing = np.vstack((perf_lab, edge_lab, screw_lab))


    y_test = np.asarray(y_test_testing)

    # Normalize
    X_test_testing = norma2(X_test_testing)


    X_test_testing = X_test_testing.reshape((X_test_testing.shape[0], 64, 64, 1))

    X_test_testing = np.asarray(np.repeat(X_test_testing, 3, 3))


    actual_vals = []

    for v in y_test:
        if all(v == [1,0,0]):
            actual_vals.append(0)

        elif all(v == [0,1,0]):
            actual_vals.append(1)

        elif all(v == [0,0,1]):
            actual_vals.append(2)


    preds = model.predict(X_test_testing)

    from sklearn.metrics import confusion_matrix

    pred_vals = []

    for v2 in preds:
        pred_vals.append(np.argmax(v2))


    def percents(conf):
        ars = []
        for idx, ar in enumerate(conf):
            total = np.sum(ar)

            output = ar[idx]/total
            #print(output)
            ars.append(output)

        return ars

    conf = confusion_matrix(actual_vals, pred_vals, labels= (0, 1, 2))
    print(conf)
    coos = percents(conf)
    print(coos)

[[3456    0    0]
 [  12 2734  710]
 [   0 1731 1725]]
[1.0, 0.7910879629629629, 0.4991319444444444]
[[3426    0   30]
 [   0 2903  553]
 [   0 1267 2189]]
[0.9913194444444444, 0.8399884259259259, 0.6333912037037037]
[[3454    0    2]
 [   5 2462  989]
 [   0 1156 2300]]
[0.9994212962962963, 0.7123842592592593, 0.6655092592592593]
